# pre lib 

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'

import warnings 
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


## 查看 dict 的 前三个内容 
import itertools
def get_first_three_from_dict(my_dict): 
    # 转换字典为迭代器
    iter_dict = iter(my_dict.items())
    # 使用 islice 取出前三个元素
    first_three = dict(itertools.islice(iter_dict, 3))
    return first_three

# data lib 

In [2]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

import glob
import pandas as pd
import json
import re 

import random 
random.seed(0)

# 文件读取和清洗的参数

In [3]:
## 读取数据的内容 
def read_train_valid_test(path): 
    data = []
    
    assert path.endswith('.txt')
    # 打开并逐行读取txt文件
    with open(path, 'r') as f:
        for line in f:
            # 使用json.loads将每一行转换为字典
            data.append(json.loads(line))
            
    ## 转换成 df 格式 
    data = pd.DataFrame(data)
    return data 

def read_json_to_df(json_path): 
    # 打开json文件
    with open(json_path, 'r') as file:
        # 解析json文件
        data = json.load(file)
    ## json --> df 
    data = pd.DataFrame(data).T.reset_index(names=['pids'])
    return data

## 清洗数据 
def clean_body_remove_symbol(text): 
    ## clean_body_remove_symbol(text) 
    text = re.sub('<[^<]+?>', ' ', text).replace('\n', '').strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('http://', '').replace('https://', '').replace('.com', '').replace('.cn', '')
    return text 

## data
trpath = 'data/AQA/qa_train.txt'
train = read_train_valid_test(trpath)

valpath = 'data/AQA/qa_valid_wo_ans.txt'
valid = read_train_valid_test(valpath)

testpath = 'data/AQA-test-public/qa_test_wo_ans_new.txt'
test = read_train_valid_test(testpath)

## json  
json_path = 'data/AQA/pid_to_title_abs_new.json'
df_json_old = read_json_to_df(json_path) 

json_path = 'data/AQA-test-public/pid_to_title_abs_update_filter.json'
df_json_new = read_json_to_df(json_path) 

len(set(df_json_new['pids']).difference(set(df_json_old['pids'])))
df_json = pd.merge(df_json_new, df_json_old, how='outer', on=['pids', 'title', 'abstract'])

del df_json_new, df_json_old

##
train.isnull().sum()
valid.isnull().sum()
df_json.isnull().sum()

## train 中的 body 内容给定 
train['body'] = train['body'].apply(clean_body_remove_symbol)
valid['body'] = valid['body'].apply(clean_body_remove_symbol)
test['body'] = test['body'].apply(clean_body_remove_symbol)

## passage 文章清洗 
df_json['title'] = df_json['title'].fillna('None').apply(clean_body_remove_symbol) 
df_json['abstract'] = df_json['abstract'].apply(clean_body_remove_symbol) 

df_json = df_json.reset_index()
df_json = df_json.rename(columns={'index':'id'}) 

70575

question    0
body        0
pids        0
dtype: int64

question    0
body        0
dtype: int64

pids        0
title       3
abstract    0
dtype: int64

In [4]:
train.head(2)
valid.head(2)
test.head(2)
df_json.head(2)

,question,body,pids
0,Why would it ever be possible for Java to be f...,Sometimes Java outperforms C++ in benchmarks. ...,[619bb02b1c45e57ce901d5f1]
1,Are stacks the only reasonable way to structur...,Most architectures I've seen rely on a call st...,[53e99876b7602d97020b053d]


,question,body
0,How is this Pytorch expression equivalent to t...,I found the following PyTorch code (from this ...
1,Why do universities have to spend money on jou...,Obviously this is a question in the light of t...


,question,body
0,Cloud Computing - Suggesting customers migrate...,AN OVERBROAD PATENT ON suggesting customer mig...
1,Requesting prior art on Google machine learnin...,Google is attempting to patent well known conc...


,id,pids,title,abstract
0,0,5390877920f70186a0d2cb29,A New Use Of An Automated Reasoning Assistant ...,The field of automated reasoning is an outgrow...
1,1,5390877920f70186a0d2cc14,Why AM an EUISKO appear to work.,"Seven years ago, the AM program was constructe..."


# Create candidates - Retrival for Training and Valid

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,2'

import ctypes
import gc
import torch
import faiss
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from torch.utils.data import DataLoader
from transformers import (
    BatchEncoding, AutoModelForMultipleChoice,
    AutoTokenizer, PreTrainedTokenizer,
    AutoModelForSequenceClassification,
    AutoModel, PreTrainedModel,
    AutoConfig, AutoModelForCausalLM
)
from sentence_transformers import SentenceTransformer
from accelerate import init_empty_weights
from accelerate.utils.modeling import set_module_tensor_to_device
from safetensors.torch import load_file

## 用于 topK 生成 
import math
import pickle
import multiprocessing

from typing import List, Tuple, Dict, Union
from concurrent.futures import ThreadPoolExecutor

## add 
import time
import itertools

## 
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

In [6]:
os.makedirs('outslgb', exist_ok=True)

## MiniLM-6

### encode question 

In [7]:
def encode_questions_minilm(args):
    """Using all-MiniLM-L6-v2 model,
    Generate embeddings for concatenation of prompt and answer options.
    It is supposed to run in 2 processes, each embeds half of the test dataframe.
    Result as float16 numpy array [len(df) x 384] is saved in pkl file.

    Args:
        args:  
            minilm_name (str) 
            proc_id (int): number of process we are in.
    """
    minilm_name, proc_id, df = args
    
    if minilm_name == 'minilmL12': 
        model_name = '/mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L12-v2/'
    elif minilm_name == 'minilm': 
        model_name = '/mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L6-v2/'
    elif minilm_name == 'paraphrase': 
        model_name = '/mntdata/wangql43/A000Files/A003Model/bensonpeng/paraphrase-MiniLM-L12-v2/'
        
    model = SentenceTransformer(model_name, trust_remote_code=True)
    
    # valpath = 'data/AQA/qa_valid_wo_ans.txt'
    # df = read_train_valid_test(valpath)
    # df['body'] = df['body'].apply(clean_body_remove_symbol)
    
    df = np.array_split(df, 2)[proc_id]
    texts = []
    for _, row in df.iterrows():
        text = f"{row.question}\n{row.body}"
        texts.append(text)
    embs = model.encode(texts, device=f'cuda:{proc_id}', batch_size=256).astype(np.float16)
    
    with open(f'outslgb/encoded_questions_{minilm_name}_{df_name}_{proc_id}.pkl', 'wb') as f:
        pickle.dump(embs, f)

### combine question embeddings 

In [8]:
def combine_embs_from_processes(model_name: str):
    """Question encoding functions produce two embeddings files for each model,
    this one combines two into one. 
    Args:
        model_name (str): 'bge' or 'minilm'.
    """
    embs = []
    for proc_idx in range(2):
        with open(f'outslgb/encoded_questions_{model_name}_{df_name}_{proc_idx}.pkl', 'rb') as f:
            embs.append(pickle.load(f))
    embs = np.concatenate(embs, axis=0)
    with open(f'outslgb/encoded_questions_{model_name}_{df_name}.pkl', 'wb') as f:
        pickle.dump(embs, f) 

### test

In [9]:
# # MiniLM-L6-L12-paraphrase 
start = time.time()
global df_name
df_name = 'test'

params_list = list(itertools.product(['minilmL12', 'minilm', 'paraphrase'], [0,1], [test]))
with multiprocessing.Pool(len(params_list)) as pool:
    pool.map(encode_questions_minilm, params_list) 
    
with multiprocessing.Pool(3) as pool:
    pool.map(combine_embs_from_processes, [('minilmL12'), ('minilm'), ('paraphrase')])
    
print(f'MiniLM-L6-L12-paraphrase-v2 encoding: {time.time() - start} s')
del df_name

Some weights of the model checkpoint at /mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L6-v2/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L6-v2/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFor

[None, None, None, None, None, None]

[None, None, None]

MiniLM-L6-L12-paraphrase-v2 encoding: 12.403230905532837 s


### valid 

In [10]:
# # # MiniLM-L6-L12-paraphrase 
# start = time.time()
# global df_name
# df_name = 'valid'

# params_list = list(itertools.product(['minilmL12', 'minilm', 'paraphrase'], [0,1], [valid]))
# with multiprocessing.Pool(len(params_list)) as pool:
#     pool.map(encode_questions_minilm, params_list) 
    
# with multiprocessing.Pool(3) as pool:
#     pool.map(combine_embs_from_processes, [('minilmL12'), ('minilm'), ('paraphrase')])
    
# print(f'MiniLM-L6-L12-paraphrase-v2 encoding: {time.time() - start} s')
# del df_name

### train 

In [11]:
# # # MiniLM-L6-L12-paraphrase 
# start = time.time()
# global df_name
# df_name = 'train'

# params_list = list(itertools.product(['minilmL12', 'minilm', 'paraphrase'], [0,1], [train]))
# with multiprocessing.Pool(len(params_list)) as pool:
#     pool.map(encode_questions_minilm, params_list) 
    
# with multiprocessing.Pool(3) as pool:
#     pool.map(combine_embs_from_processes, [('minilmL12'), ('minilm'), ('paraphrase')])
    
# print(f'MiniLM-L6-L12-paraphrase-v2 encoding: {time.time() - start} s')
# del df_name

## bge + gte 

### encode question 

In [12]:
def encode_questions_bge(args):
    """Using BAAI/bge-small-en-v1.5 model,
    Generate embeddings for concatenation of prompt and answer options.
    It is supposed to run in 2 processes, each embeds half of the test dataframe.
    Result as float16 numpy array [len(df) x 384] is saved in pkl file.

    Args:
        proc_id (int): number of process we are in.
    """
    proc_id, df = args
    
    if bge_name == 'bge': 
        model_path = '/mntdata/wangql43/A000Files/A003Model/AI-ModelScope/bge-large-en-v1.5/'
    elif bge_name == 'gte': 
        model_path = '/mntdata/wangql43/A000Files/A003Model/iic/nlp_gte_sentence-embedding_english-large/'
    elif bge_name == 'bgeM3':
        model_path = '/mntdata/wangql43/A000Files/A003Model/AI-ModelScope/bge-m3/'
    elif bge_name == 'bgeLarge':
        model_path = '/mntdata/wangql43/A000Files/A003Model/AI-ModelScope/bge-reranker-large/'
        # model_path = 'models/mixed_model_1/'
        # model_path = '/data/wangql43/A000Files/A000comp/006kdd/AQA-KDD-2024/RAG-Retrieval/rag_retrieval/train/embedding/output/test_trainingModel/model/'
        
        
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    model.eval()
    model.to(f'cuda:{proc_id}')

    # valpath = 'data/AQA/qa_valid_wo_ans.txt'
    # df = read_train_valid_test(valpath)
    # df['body'] = df['body'].apply(clean_body_remove_symbol)

    df = np.array_split(df, 2)[proc_id]
    texts = []
    instr = "Represent this sentence for searching relevant passages: "
    for _, row in df.iterrows():
        colon = ':'
        if row.question.endswith(':'):
            colon = ''
        text = f"{instr}{row.question}{colon} {row.body}"
        texts.append(text)

    embeddings = []
    dataloader = DataLoader(
        texts, batch_size=32, num_workers=0,
        collate_fn=lambda batch: tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
    )
    with torch.no_grad():
        with torch.autocast(device_type='cuda'):
            for batch in tqdm(dataloader):
                model_output = model(**batch.to(model.device))
                sentence_embeddings = model_output[0][:, 0]
                sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
                embeddings.append(sentence_embeddings)
    embeddings = torch.cat(embeddings, dim=0).cpu().numpy().astype(np.float16)

    with open(f'outslgb/encoded_questions_{bge_name}_{df_name}_{proc_id}.pkl', 'wb') as f:
        pickle.dump(embeddings, f) 
    

### test

In [13]:
## bge-gte
start = time.time()

global bge_name, df_name
df_name = 'test'
params_list = list(itertools.product([0,1], [test]))

bge_name = 'bge'
with multiprocessing.Pool(2) as pool:
    pool.map(encode_questions_bge, params_list) 
    
bge_name = 'gte'
with multiprocessing.Pool(2) as pool:
    pool.map(encode_questions_bge, params_list) 
    
bge_name = 'bgeM3'
with multiprocessing.Pool(2) as pool:
    pool.map(encode_questions_bge, params_list) 

del bge_name; clean_memory()

with multiprocessing.Pool(1) as pool:
    pool.map(combine_embs_from_processes, ['bge', 'gte', 'bgeM3'])

print(f'bge-gte encoding: {time.time() - start} s') 

del df_name

[None, None]

[None, None]

[None, None]

[None, None, None]

bge-gte encoding: 49.956908226013184 s


### valid

In [14]:
# # bge-gte
# start = time.time()

# global bge_name, df_name
# df_name = 'valid'
# params_list = list(itertools.product([0,1], [valid]))

# bge_name = 'bge'
# with multiprocessing.Pool(2) as pool:
#     pool.map(encode_questions_bge, params_list) 
    
# bge_name = 'gte'
# with multiprocessing.Pool(2) as pool:
#     pool.map(encode_questions_bge, params_list) 
    
# del bge_name; clean_memory()

# with multiprocessing.Pool(1) as pool:
#     pool.map(combine_embs_from_processes, ['bge', 'gte'])

# print(f'bge-gte encoding: {time.time() - start} s') 

# del df_name

### train

In [15]:
# # bge-gte
# start = time.time()

# global bge_name, df_name
# df_name = 'train'
# params_list = list(itertools.product([0,1], [train]))

# bge_name = 'bge'
# with multiprocessing.Pool(2) as pool:
#     pool.map(encode_questions_bge, params_list) 
    
# bge_name = 'gte'
# with multiprocessing.Pool(2) as pool:
#     pool.map(encode_questions_bge, params_list) 
    
# del bge_name; clean_memory()

# with multiprocessing.Pool(1) as pool:
#     pool.map(combine_embs_from_processes, ['bge', 'gte'])

# print(f'bge-gte encoding: {time.time() - start} s') 

# del df_name

## passage-embedding 

In [16]:
### minilm embedding 
def encode_passageJson_minilm(args):
    """Using all-MiniLM-L6-v2 model,
    Generate embeddings for concatenation of prompt and answer options.
    It is supposed to run in 2 processes, each embeds half of the test dataframe.
    Result as float16 numpy array [len(df) x 384] is saved in pkl file.

    Args:
        proc_id (int): number of process we are in.
    """
    minilm_name, proc_id, df = args
    
    if minilm_name == 'minilmL12': 
        model_name = '/mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L12-v2/'
    elif minilm_name == 'minilm': 
        model_name = '/mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L6-v2/'
    elif minilm_name == 'paraphrase': 
        model_name = '/mntdata/wangql43/A000Files/A003Model/bensonpeng/paraphrase-MiniLM-L12-v2/'
        
    model = SentenceTransformer(model_name, trust_remote_code=True)
    
    ## encode 
    df = np.array_split(df, 2)[proc_id]
    texts = []
    for _, row in df.iterrows():
        text = f"{row.title}\n{row.abstract}"
        texts.append(text)
    embs = model.encode(texts, device=f'cuda:{proc_id}', batch_size=256).astype(np.float16)
    with open(f'outslgb/encoded_passageJson_{minilm_name}_{proc_id}.pkl', 'wb') as f:
        pickle.dump(embs, f)
        

### bge embedding 
def encode_passageJson_bge(args):
    """Using BAAI/bge-small-en-v1.5 model,
    Generate embeddings for concatenation of prompt and answer options.
    It is supposed to run in 2 processes, each embeds half of the test dataframe.
    Result as float16 numpy array [len(df) x 384] is saved in pkl file.

    Args:
        proc_id (int): number of process we are in.
    """
    proc_id, df = args
    
    if bge_name == 'bge': 
        model_path = '/mntdata/wangql43/A000Files/A003Model/AI-ModelScope/bge-large-en-v1.5/'
    elif bge_name == 'gte': 
        model_path = '/mntdata/wangql43/A000Files/A003Model/iic/nlp_gte_sentence-embedding_english-large/'
    elif bge_name == 'bgeM3':
        model_path = '/mntdata/wangql43/A000Files/A003Model/AI-ModelScope/bge-m3/'
    elif bge_name == 'bgeLarge':
        model_path = '/mntdata/wangql43/A000Files/A003Model/AI-ModelScope/bge-reranker-large/'
        # model_path = 'models/mixed_model_1/'
        # model_path = '/data/wangql43/A000Files/A000comp/006kdd/AQA-KDD-2024/RAG-Retrieval/rag_retrieval/train/embedding/output/test_trainingModel/model/'
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path, trust_remote_code=True) 
    model.eval()
    model.to(f'cuda:{proc_id}')
    
    ## 读取 json 文件
    df = np.array_split(df, 2)[proc_id]
    texts = []
    instr = "Represent this sentence for searching relevant passages: "
    for _, row in df.iterrows():
        colon = ':'
        if row.title.endswith(':'):
            colon = ''
        text = f"{instr}{row.title}{colon} {row.abstract}"
        texts.append(text)

    embeddings = []
    dataloader = DataLoader(
        texts, batch_size=128, num_workers=0,
        collate_fn=lambda batch: tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')
    )
    with torch.no_grad():
        with torch.autocast(device_type='cuda'):
            for batch in tqdm(dataloader):
                model_output = model(**batch.to(model.device))
                sentence_embeddings = model_output[0][:, 0]
                sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
                embeddings.append(sentence_embeddings)
    embeddings = torch.cat(embeddings, dim=0).cpu().numpy().astype(np.float16)

    with open(f'outslgb/encoded_passageJson_{bge_name}_{proc_id}.pkl', 'wb') as f:
        pickle.dump(embeddings, f)

### 
def combine_passageJson_embs_from_processes(model_name: str):
    """Question encoding functions produce two embeddings files for each model,
    this one combines two into one. 

    Args:
        model_name (str): 'bge' or 'minilm'.
    """
    embs = []
    for proc_idx in range(2):
        with open(f'outslgb/encoded_passageJson_{model_name}_{proc_idx}.pkl', 'rb') as f:
            embs.append(pickle.load(f))
    embs = np.concatenate(embs, axis=0)
    with open(f'outslgb/encoded_passageJson_{model_name}.pkl', 'wb') as f:
        pickle.dump(embs, f) 

### passage - minilm 

In [17]:
start = time.time()
params_list = list(itertools.product(['minilmL12', 'minilm', 'paraphrase'], [0,1], [df_json]))
with multiprocessing.Pool(len(params_list)) as pool:
    pool.map(encode_passageJson_minilm, params_list) 

with multiprocessing.Pool(3) as pool:
    pool.map(combine_passageJson_embs_from_processes, [('minilmL12'), ('minilm'), ('paraphrase')])
print(f'MiniLM-L6-L12-paraphrase-v2 encoding: {time.time() - start} s')

Some weights of the model checkpoint at /mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L6-v2/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /mntdata/wangql43/A000Files/A003Model/bensonpeng/all-MiniLM-L6-v2/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFor

[None, None, None, None, None, None]

[None, None, None]

MiniLM-L6-L12-paraphrase-v2 encoding: 719.149254322052 s


### passage - gte + bge 

In [18]:
start = time.time()
## 
params_list = list(itertools.product([0,1], [df_json]))

global bge_name
bge_name = 'bge'
with multiprocessing.Pool(2) as pool:
    pool.map(encode_passageJson_bge, params_list) 
    
bge_name = 'gte'
with multiprocessing.Pool(2) as pool:
    pool.map(encode_passageJson_bge, params_list) 
    
bge_name = 'bgeM3'
with multiprocessing.Pool(2) as pool:
    pool.map(encode_passageJson_bge, params_list) 
    
del bge_name; clean_memory()

with multiprocessing.Pool(1) as pool:
    pool.map(combine_passageJson_embs_from_processes, ['bge', 'gte', 'bgeM3'])

# with multiprocessing.Pool(1) as pool:
#     pool.map(combine_passageJson_embs_from_processes, ['gte'])
    
print(f'bge-large encoding: {time.time() - start} s') 

[None, None]

[None, None]

[None, None]

[None, None, None]

bge-large encoding: 4632.649368047714 s


### passage idx 

In [19]:
## 给passages信息加上 id的映射 
# df_json['id'] = df_json.index

## id2pids
id2pids_dict = dict(zip(df_json['id'], df_json['pids']))
with open(f'outslgb/id2pids_dict.pkl', 'wb') as f:
    pickle.dump(id2pids_dict, f) 

## pids2id
pids2id_dict = dict(zip(df_json['pids'], df_json['id']))
with open(f'outslgb/pids2id_dict.pkl', 'wb') as f:
    pickle.dump(pids2id_dict, f) 

## 每个 model 都来一对 query2passage topK passages 

In [20]:
def calculate_similarities(
    model_name: str,
    device: Union[str, int], 
    df_name: str
) -> torch.Tensor:
    '''
    minilm_similarities = calculate_similarities(model_name='minilm', device='cuda:0', train)
    '''
    
    ## 读取 query 的 embedding 内容 
    with open(f'outslgb/encoded_questions_{model_name}_{df_name}.pkl', 'rb') as f:
        query_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)
        
    ## passage 的 embedding 内容 
    passages_embs_dir = {
        ## sentence model 
        'minilm': 'outslgb/encoded_passageJson_minilm.pkl',
        'minilmL12': 'outslgb/encoded_passageJson_minilmL12.pkl',
        'paraphrase': 'outslgb/encoded_passageJson_paraphrase.pkl',
        ## model 
        'bge': 'outslgb/encoded_passageJson_bge.pkl',
        'gte': 'outslgb/encoded_passageJson_gte.pkl',
        'bgeM3': 'outslgb/encoded_passageJson_bgeM3.pkl',
        # 'bgeLarge': 'outslgb/encoded_passageJson_bgeLarge.pkl',
    }[model_name]
    with open(passages_embs_dir, 'rb') as f:
        passages_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)
        
    clean_memory()
    
    ## 直接计算两者的相似度 
    similarity = query_embs @ passages_embs.T
    return similarity


### 
def get_query_passages_similarities(df_name):
    # Calculate similarities
    args = [('minilm', 'cuda:0', df_name), 
            ('bge', 'cuda:1', df_name), 
            ('minilmL12', 'cuda:0', df_name), 
            ('gte', 'cuda:1', df_name)
           ]
    with ThreadPoolExecutor() as executor:
        minilm_similarities, bge_similarities, minilm_similarities_l12, gte_similarities = tuple(executor.map(lambda p: calculate_similarities(*p), args))
    clean_memory()
    
    return minilm_similarities, bge_similarities, minilm_similarities_l12, gte_similarities


### 
def get_query_passages_similarities_addOther(df_name):
    # Calculate similarities
    args = [
            ('paraphrase', 'cuda:0', df_name), 
            ('bgeM3', 'cuda:1', df_name), 
           ]
    with ThreadPoolExecutor() as executor:
        paraphrase_similarities, bgeM3_similarities = tuple(executor.map(lambda p: calculate_similarities(*p), args))
    clean_memory()
    
    return paraphrase_similarities, bgeM3_similarities

In [21]:
def get_top_passages_ids(minilm_similarities):
    ### 
    n_queries = minilm_similarities.shape[0]
    batch_size = 100
    n_batches = math.ceil(n_queries/batch_size)

    ## 
    top_indices, top_similarities = [], []
    for i in range(n_batches):
        start = i*batch_size
        end = (i+1)*batch_size
        b_top_indices = torch.argsort(minilm_similarities[start:end], dim=1, descending=True)
        b_top_similarities = torch.take_along_dim(minilm_similarities[start:end], b_top_indices, dim=1)[:, :N_PASSAGES].cpu().numpy()
        b_top_indices = b_top_indices[:, :N_PASSAGES].cpu().numpy()
        top_indices.append(b_top_indices)
        top_similarities.append(b_top_similarities)
    top_similarities = np.concatenate(top_similarities, axis=0)
    top_indices = np.concatenate(top_indices, axis=0)
    
    ## 输出 相似度分数 和 top_indices 
    return top_similarities, top_indices


In [22]:
def save_similar_indices(train_valid, top_similarities, top_indices, model_name): 
    ## id2pids_dict 映射表 
    with open('outslgb/id2pids_dict.pkl', 'rb') as f:
        id2pids_dict = pickle.load(f) 
        
    ## top-indies 
    top_indices_pids = [[id2pids_dict[value] for value in sublist] for sublist in top_indices] 
    top_indices_pids = [','.join(subist) for subist in top_indices_pids]

    ## top-similar
    top_similarities_pids_score = [','.join([str(value) for value in subist]) for subist in top_similarities]
    
    ## 返回输出内容
    train_valid[f'retrival_pids_{model_name}'] = top_indices_pids
    train_valid[f'retrival_pids_scores_{model_name}'] = top_similarities_pids_score 
    
    return train_valid

In [23]:
# train_minilm_similarities, train_bge_similarities, train_minilm_similarities_l12, train_gte_similarities = get_query_passages_similarities('train')
# valid_minilm_similarities, valid_bge_similarities, valid_minilm_similarities_l12, valid_gte_similarities = get_query_passages_similarities('valid')

In [24]:
test_minilm_similarities, test_bge_similarities, test_minilm_similarities_l12, test_gte_similarities = get_query_passages_similarities('test')
test_paraphrase_similarities, test_bgeM3_similarities = get_query_passages_similarities_addOther('test')

In [25]:
# N_PASSAGES = 300  ## 0.18366 llm-merged 
# N_PASSAGES = 200  ## 0.1833
# N_PASSAGES = 100    ## 0.1836
N_PASSAGES = 150  ## 0.18386 llm-merged 


def get_candidates_for_train_valid(train_valid, top_similarities, model_name='minilmL6', symbol='train'): 
    model_name_top_similarities, model_name_top_indices = get_top_passages_ids(top_similarities)
    df_model_name_candidates = save_similar_indices(train_valid, model_name_top_similarities, model_name_top_indices, model_name)
    df_model_name_candidates.to_parquet(f'outslgb/{symbol}_{model_name}_topK{N_PASSAGES}_candidates.parquet', index=False) 
    return df_model_name_candidates

### minilm-L6 

In [26]:
# train_minilmL6_candidates = get_candidates_for_train_valid(train, train_minilm_similarities, model_name='minilmL6', symbol='train')

# valid_minilmL6_candidates = get_candidates_for_train_valid(valid, valid_minilm_similarities, model_name='minilmL6', symbol='valid')

# del train_minilmL6_candidates, valid_minilmL6_candidates

In [27]:
test_minilmL6_candidates = get_candidates_for_train_valid(test, test_minilm_similarities, model_name='minilmL6', symbol='test')

### minilm-L12 

In [28]:
# train_minilmL12_candidates = get_candidates_for_train_valid(train, train_minilm_similarities_l12, model_name='minilmL12', symbol='train')

# valid_minilmL12_candidates = get_candidates_for_train_valid(valid, valid_minilm_similarities_l12, model_name='minilmL12', symbol='valid')

# del train_minilmL12_candidates, valid_minilmL12_candidates

In [29]:
test_minilmL12_candidates = get_candidates_for_train_valid(test, test_minilm_similarities_l12, model_name='minilmL12', symbol='test')

### bge 

In [30]:
# train_bge_candidates = get_candidates_for_train_valid(train, train_bge_similarities, model_name='bge', symbol='train')

# valid_bge_candidates = get_candidates_for_train_valid(valid, valid_bge_similarities, model_name='bge', symbol='valid')

# del train_bge_candidates, valid_bge_candidates

In [31]:
test_bge_candidates = get_candidates_for_train_valid(test, test_bge_similarities, model_name='bge', symbol='test')

### gte

In [32]:
# train_gte_candidates = get_candidates_for_train_valid(train, train_gte_similarities, model_name='gte', symbol='train')

# valid_gte_candidates = get_candidates_for_train_valid(valid, valid_gte_similarities, model_name='gte', symbol='valid')

# del train_gte_candidates, valid_gte_candidates

In [33]:
test_gte_candidates = get_candidates_for_train_valid(test, test_gte_similarities, model_name='gte', symbol='test')

### paraphrase

In [34]:
test_paraphrase_candidates = get_candidates_for_train_valid(test, test_paraphrase_similarities, model_name='paraphrase', symbol='test')

### bgeM3

In [35]:
test_bgeM3_candidates = get_candidates_for_train_valid(test, test_bgeM3_similarities, model_name='bgeM3', symbol='test')

### bgeLarge

In [36]:
### 不行 
### test_bgeLarge_candidates = get_candidates_for_train_valid(test, test_bgeLarge_similarities, model_name='bgeLarge', symbol='test')

### 清理内存占用

In [37]:
train.head(2)

,question,body,pids
0,Why would it ever be possible for Java to be f...,Sometimes Java outperforms C++ in benchmarks. ...,[619bb02b1c45e57ce901d5f1]
1,Are stacks the only reasonable way to structur...,Most architectures I've seen rely on a call st...,[53e99876b7602d97020b053d]


In [38]:
# del train_minilm_similarities, train_bge_similarities, train_minilm_similarities_l12, train_gte_similarities
# del valid_minilm_similarities, valid_bge_similarities, valid_minilm_similarities_l12, valid_gte_similarities

In [39]:
clean_memory()

In [40]:
N_PASSAGES
test_bgeM3_candidates = pd.read_parquet(f'outslgb/test_bgeM3_topK{N_PASSAGES}_candidates.parquet')

150

In [41]:
test_bgeM3_candidates.head(2)

,question,body,retrival_pids_minilmL6,retrival_pids_scores_minilmL6,retrival_pids_minilmL12,retrival_pids_scores_minilmL12,retrival_pids_bge,retrival_pids_scores_bge,retrival_pids_gte,retrival_pids_scores_gte,retrival_pids_paraphrase,retrival_pids_scores_paraphrase,retrival_pids_bgeM3,retrival_pids_scores_bgeM3
0,Cloud Computing - Suggesting customers migrate...,AN OVERBROAD PATENT ON suggesting customer mig...,"5390ac1820f70186a0eb4898,558bf479e4b02b9f07a43...","0.5815,0.5796,0.5737,0.5728,0.558,0.5503,0.548...","53e9b381b7602d9703e63436,5390b1d220f70186a0ee2...","0.6074,0.589,0.581,0.567,0.5513,0.539,0.5244,0...","5390ac1820f70186a0eb4898,5390b1d220f70186a0ee2...","0.754,0.7295,0.7026,0.6987,0.6963,0.6963,0.692...","5390ac1820f70186a0eb4898,64d641fe3fda6d7f06226...","0.8975,0.892,0.8867,0.877,0.877,0.877,0.874,0....","5390ac1820f70186a0eb4898,53e99a4eb7602d97022b1...","7.133,7.03,6.906,6.906,6.785,6.625,6.54,6.406,...","53e9a797b7602d97030d0e95,5390b1d220f70186a0ee2...","0.7544,0.7266,0.7104,0.707,0.705,0.705,0.7026,..."
1,Requesting prior art on Google machine learnin...,Google is attempting to patent well known conc...,"53e9ac5bb7602d970362a4f8,53e99d1ab7602d97025cd...","0.5146,0.508,0.4763,0.4714,0.4663,0.4648,0.459...","53e9a26bb7602d9702b72b1d,598d17fb0cf2ddbbee17d...","0.4414,0.428,0.427,0.4187,0.4028,0.3992,0.3965...","53e9a26bb7602d9702b72b1d,5d9edc1947c8f76646032...","0.685,0.6826,0.6777,0.674,0.665,0.664,0.6626,0...","53e9a26bb7602d9702b72b1d,60c2be866750f85387880...","0.8735,0.8696,0.867,0.8657,0.863,0.8623,0.862,...","5e60d4a093d709897cce6ce6,5d9edc1947c8f76646032...","7.223,6.84,6.793,6.766,6.656,6.46,6.46,6.42,6....","5c7569e8f56def97982e75bf,645647a5d68f896efae34...","0.699,0.6943,0.694,0.689,0.688,0.6875,0.687,0...."


### 构建结果数据集 

In [42]:
# test_bgeM3_candidates['pids2score_minilmL6_dict'] = test_bgeM3_candidates.apply(lambda row: dict(zip(row['retrival_pids_minilmL6'].split(','), row['retrival_pids_scores_minilmL6'].split(','))), axis=1)
# # test_bgeM3_candidates['pids2score_minilmL12_dict'] = test_bgeM3_candidates.apply(lambda row: dict(zip(row['retrival_pids_minilmL12'].split(','), row['retrival_pids_scores_minilmL12'].split(','))), axis=1)
# # test_bgeM3_candidates['pids2score_bge_dict'] = test_bgeM3_candidates.apply(lambda row: dict(zip(row['retrival_pids_bge'].split(','), row['retrival_pids_scores_bge'].split(','))), axis=1)
# # test_bgeM3_candidates['pids2score_gte_dict'] = test_bgeM3_candidates.apply(lambda row: dict(zip(row['retrival_pids_gte'].split(','), row['retrival_pids_scores_gte'].split(','))), axis=1)
# # test_bgeM3_candidates['pids2score_paraphrase_dict'] = test_bgeM3_candidates.apply(lambda row: dict(zip(row['retrival_pids_paraphrase'].split(','), row['retrival_pids_scores_paraphrase'].split(','))), axis=1)
# # test_bgeM3_candidates['pids2score_bgeM3_dict'] = test_bgeM3_candidates.apply(lambda row: dict(zip(row['retrival_pids_bgeM3'].split(','), row['retrival_pids_scores_bgeM3'].split(','))), axis=1)

# cols = [col for col in test_bgeM3_candidates.columns if ('score' not in col) and (col not in ['question', 'body'])]
# for col in cols: 
#     test_bgeM3_candidates[col] = test_bgeM3_candidates[col].str.split(',')

# test_minilmL6_candidates = test_bgeM3_candidates[['question', 'body', 'retrival_pids_minilmL6', 'pids2score_minilmL6_dict']].explode(column='retrival_pids_minilmL6')
# # test_minilmL12_candidates = test_bgeM3_candidates[['question', 'body', 'retrival_pids_minilmL12', 'pids2score_minilmL12_dict']].explode(column='retrival_pids_minilmL12')
# # test_bge_candidates = test_bgeM3_candidates[['question', 'body', 'retrival_pids_bge', 'pids2score_bge_dict']].explode(column='retrival_pids_bge')
# # test_gte_candidates = test_bgeM3_candidates[['question', 'body', 'retrival_pids_gte', 'pids2score_gte_dict']].explode(column='retrival_pids_gte')
# # test_paraphrase_candidates = test_bgeM3_candidates[['question', 'body', 'retrival_pids_paraphrase', 'pids2score_paraphrase_dict']].explode(column='retrival_pids_paraphrase')
# # test_bgeM3_candidates_short = test_bgeM3_candidates[['question', 'body', 'retrival_pids_bgeM3', 'pids2score_bgeM3_dict']].explode(column='retrival_pids_bgeM3')

## ranker-candidates 构建

In [43]:
## 先暂时忽略顺序 
def construct_candidates(row): 
    return list(set((row['retrival_pids_minilmL6'] + ',' + \
                     row['retrival_pids_minilmL12'] + ',' + \
                     row['retrival_pids_bge'] + ',' + \
                     row['retrival_pids_gte'] + ',' + \
                     row['retrival_pids_paraphrase'] + ',' + \
                     row['retrival_pids_bgeM3']                     
                    ).split(',')))

# train['candidates'] = train.apply(construct_candidates, axis=1)
# valid['candidates'] = valid.apply(construct_candidates, axis=1)
test['candidates'] = test_bgeM3_candidates.apply(construct_candidates, axis=1) 
test['candidates'].apply(lambda x: len(x)).describe()
test.to_parquet('outslgb/test_remove_pids_from_candidates.parquet', index=False) 

count    3000.000000
mean      557.904667
std        92.519938
min       263.000000
25%       494.000000
50%       559.000000
75%       621.000000
max       832.000000
Name: candidates, dtype: float64

In [45]:
## 使用 set 求差值 （移除掉召回正确的选项）
def remove_pids_from_candidates(row): 
    return list(set(row['candidates']).difference(set(row['pids'])))

# train['neg_candidates'] = train.apply(remove_pids_from_candidates, axis=1)
# if 'neg_candidates' in train.columns: 
#     print(1)

# if 'neg_candidates' in train.columns: 
#     train.to_parquet('outslgb/train_remove_pids_from_candidates.parquet', index=False)
#     valid.to_parquet('outslgb/valid_remove_pids_from_candidates.parquet', index=False)
#     test.to_parquet('outslgb/test_remove_pids_from_candidates.parquet', index=False) 

# train = pd.read_parquet('outslgb/train_remove_pids_from_candidates.parquet')
# valid = pd.read_parquet('outslgb/valid_remove_pids_from_candidates.parquet')

In [46]:
test = pd.read_parquet('outslgb/test_remove_pids_from_candidates.parquet')

In [47]:
# df_pos_train = train.explode('pids', ignore_index=False)[['question', 'body', 'pids']]
# df_pos_train['label'] = int(1) 
# df_neg_train = train.explode('neg_candidates', ignore_index=False)[['question', 'body', 'neg_candidates']]
# df_neg_train['label'] = int(0)
# df_neg_train = df_neg_train.rename(columns={'neg_candidates': 'pids'})
# df_train = pd.concat([df_pos_train, df_neg_train], ignore_index=False)
# df_train = df_train.sort_index().reset_index()
# # del df_pos_train, df_neg_train

# df_valid = valid.rename(columns={'candidates': 'pids'})[['question', 'body', 'pids']].explode('pids', ignore_index=False)
# df_valid = df_valid.sort_index().reset_index()
# df_valid['label'] = int(-1)

In [48]:
df_test = test.rename(columns={'candidates': 'pids'})[['question', 'body', 'pids']].explode('pids', ignore_index=False)
df_test = df_test.sort_index().reset_index()
df_test['label'] = int(-1)

In [49]:
# df_json.head(1)
# df_train = df_train.merge(df_json, how='left', on='pids')
# df_valid = df_valid.merge(df_json, how='left', on='pids')

In [50]:
df_test = df_test.merge(df_json, how='left', on='pids')

## lgb-reranker 构建特征工程 

In [51]:
from rapidfuzz import fuzz
import jellyfish  ## https://jamesturk.github.io/jellyfish/

from multiprocessing import Pool, cpu_count
from functools import partial

### 规则特征工程 

In [52]:
## 使用规则的方式计算距离 
def parallel_apply(df, func, num_threads):
    '''
    # return parallel_apply(df, partial(calculate_features, alist=alist), num_threads)    
    '''
    df_split = np.array_split(df, num_threads)
    pool = Pool(num_threads)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# 为了在多进程中使用tqdm，我们需要一个自定义的封装器
def tqdm_parallel_map(func, iterable, **kwargs):
    with Pool(cpu_count()) as pool:
        return list(tqdm(pool.imap(func=func, iterable=iterable), total=len(iterable), **kwargs))

def calculate_features(df, alist):
    cols = ['question', 'body', 'title', 'abstract']

    ## 字段列的长度 
    # for col in cols: 
    #     df[f'{col}_length'] = df[col].str.split(' ').str.len()
        
    ## 去重之后，词级别的占比 
    # for pair in alist: 
    #     # df[f'{pair[0]}_{pair[1]}_word_overlap'] = df.apply(lambda x: len(set(x[pair[0]].split(' ')) & set(x[pair[1]].split(' '))), axis=1)
    #     df[f'{pair[0]}_{pair[1]}_word_overlap'] = list(map(lambda x, y: len(set(x.split(' ')) & set(y.split(' '))), df[pair[0]], df[pair[1]]))

    ## fuzz 距离 
    for pair in alist: 
        # df[f'{pair[0]}_{pair[1]}_fuzzRatio'] = df.apply(lambda x: fuzz.ratio(x[pair[0]], x[pair[1]]), axis=1)
        df[f'{pair[0]}_{pair[1]}_fuzzRatio'] = [fuzz.ratio(x, y) for x, y in zip(df[pair[0]], df[pair[1]])]
        
    ## jellyfish 距离 
    for pair in alist: 
        # df[f'{pair[0]}_{pair[1]}_jaro_similarity'] = df.apply(lambda x: jellyfish.jaro_similarity(x[pair[0]], x[pair[1]]), axis=1)
        # df[f'{pair[0]}_{pair[1]}_damerau_levenshtein_distance'] = df.apply(lambda x: jellyfish.damerau_levenshtein_distance(x[pair[0]], x[pair[1]]), axis=1)
        # df[f'{pair[0]}_{pair[1]}_hamming_distance'] = df.apply(lambda x: jellyfish.hamming_distance(x[pair[0]], x[pair[1]]), axis=1)
        df[f'{pair[0]}_{pair[1]}_jaro_similarity'] = [jellyfish.jaro_similarity(x, y) for x, y in zip(df[pair[0]], df[pair[1]])]
        df[f'{pair[0]}_{pair[1]}_damerau_levenshtein_distance'] = [jellyfish.damerau_levenshtein_distance(x, y) for x, y in zip(df[pair[0]], df[pair[1]])]
        df[f'{pair[0]}_{pair[1]}_hamming_distance'] = [jellyfish.hamming_distance(x, y) for x, y in zip(df[pair[0]], df[pair[1]])]
        
    return df

def construct_feature_engineer(df):
    alist = list(itertools.product(['question', 'body'], ['title', 'abstract']))
    num_threads = 24  # 使用与CPU核心数相同数量的线程
    # 使用自定义的tqdm并行映射函数
    dfs = tqdm_parallel_map(partial(calculate_features, alist=alist), np.array_split(df, num_threads))
    return pd.concat(dfs)

In [ ]:
# %%time 
# ## .head(100)
# df_train = construct_feature_engineer(df_train) 

# %%time 
# ## .head(100)
# df_valid = construct_feature_engineer(df_valid)

# %%time 
# ## .head(100)
# df_test = construct_feature_engineer(df_test)

In [56]:
# # 计算 DataFrame 占用的内存量（默认情况下包括了索引）
# # 将字节转换为 MB
# memory_usage_in_mb = df_train.memory_usage(deep=True).sum() / (1024 ** 2)
# print(f"df_train >>> DataFrame 占用的内存为: {memory_usage_in_mb:.2f} MB")

# # 计算 DataFrame 占用的内存量（默认情况下包括了索引）
# # 将字节转换为 MB
# memory_usage_in_mb = df_valid.memory_usage(deep=True).sum() / (1024 ** 2)
# print(f"df_valid >>> DataFrame 占用的内存为: {memory_usage_in_mb:.2f} MB")

In [57]:
df_test.shape

(1673714, 8)

### unique_index 

In [58]:
unique_index = df_test['index'].unique()

device = 'cuda:0'

### 各自 embeds 特征工程 @ 相似度 -- 直接通过前期的 embedding dict 进行计算 

In [59]:
model_name_lst = ['minilm', 'minilmL12', 'bge', 'gte', 'bgeM3']

for model_name in model_name_lst: 
    print(f'model_name   >>>   {model_name}   ...   ')
    ## 读取 query 的 embedding 内容 
    with open(f'outslgb/encoded_questions_{model_name}_test.pkl', 'rb') as f:
        query_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)
    
    ## passage 的 embedding 内容 
    passages_embs_dir = {
        ## sentence model 
        'minilm': 'outslgb/encoded_passageJson_minilm.pkl',
        'minilmL12': 'outslgb/encoded_passageJson_minilmL12.pkl',
        ## 'paraphrase': 'outslgb/encoded_passageJson_paraphrase.pkl',
        ## model 
        'bge': 'outslgb/encoded_passageJson_bge.pkl',
        'gte': 'outslgb/encoded_passageJson_gte.pkl',
        'bgeM3': 'outslgb/encoded_passageJson_bgeM3.pkl',
    }[model_name]
    with open(passages_embs_dir, 'rb') as f:
        passages_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)
    # passages_embs_pids2dict = dict(zip(df_json['pids'].tolist(), passages_embs)) 
    
    ## 计算得分 
    for index in tqdm(unique_index, total=len(unique_index)): 
        retrival_pids = df_test.loc[df_test['index'] == index, 'id'].tolist() 
        scores = query_embs[index] @ passages_embs[retrival_pids].T
        df_test.loc[df_test['index'] == index, f'{model_name}_q2p_scores'] = scores.detach().cpu().numpy().astype(np.float32)

model_name   >>>   minilm   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   minilmL12   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   bge   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   gte   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   bgeM3   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

In [60]:
q2p_scores_cols = [col for col in df_test.columns if 'q2p_scores' in col]
q2p_scores_cols = ['minilm_q2p_scores',
                'minilmL12_q2p_scores',
                # 'paraphrase_q2p_scores',
                'bge_q2p_scores',
                'gte_q2p_scores',
                'bgeM3_q2p_scores']
df_test['rank_score_mean'] = df_test[q2p_scores_cols].fillna(0).mean(axis=1).astype(np.float32)

In [61]:
def get_rank_scores(row): 
    rank_score = 0.2 * row['minilm_q2p_scores'] + \
                0.2 * row['minilmL12_q2p_scores'] + \
                0.2 * row['bge_q2p_scores'] + \
                0.2 * row['gte_q2p_scores'] + \
                0.2 * row['bgeM3_q2p_scores']
    return rank_score

# 直接使用 numpy 进行向量化计算
weights = np.array([0.18, 0.16, 0.24, 0.2, 0.22])  # 权重向量
np.sum(weights)
q2p_scores_cols = ['minilm_q2p_scores',
                'minilmL12_q2p_scores',
                'bge_q2p_scores',
                'gte_q2p_scores',
                'bgeM3_q2p_scores']
# 确保数据类型正确，避免计算时出现类型错误
df_test[q2p_scores_cols] = df_test[q2p_scores_cols].astype(float) 
# 计算加权得分
df_test['rank_score_weights'] = np.sum(df_test[q2p_scores_cols].values * weights, axis=1)

1.0

In [62]:
def standardize_data(X):
    # 计算每一列的均值和标准差
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    
    # 对数据进行标准化
    standardized_X = (X - mean) / std
    
    return standardized_X

def min_max_normalization(X):
    # 确保数据类型为浮点数，以避免整数除法的问题
    X = X.astype('float64')
    
    # 计算每一列（特征）的最大值和最小值
    max_values = np.max(X, axis=0)
    min_values = np.min(X, axis=0)
    
    # 执行最大最小值归一化
    normalized_X = (X - min_values) / (max_values - min_values)
    
    return normalized_X
# df_test.groupby(by=['index'])['paraphrase_q2p_scores'].transform(min_max_normalization)

In [63]:
# ### 0.1484
# df_test = df_test.sort_values(by=['index', 'rank_score'], ascending=[True, False])
# test_candidates_list = df_test.groupby(by=['index'], as_index=True)['pids'].apply(list)

In [64]:
### 0.1486
df_test = df_test.sort_values(by=['index', 'rank_score_weights'], ascending=[True, False])
test_candidates_list_weight = df_test.groupby(by=['index'], as_index=True)['pids'].apply(list)

In [65]:
test_candidates_list = test_candidates_list_weight.copy() 
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))

test_candidates_list.to_csv('my_data.txt', sep='\t', index=False, header=None)

### concat embeds 计算得分 rank_score_concat 

In [66]:
query_embs_concat = []
passages_embs_concat = [] 

model_name_lst = ['minilm', 'minilmL12', 'bge', 'gte', 'bgeM3']
for model_name in model_name_lst: 
    print(f'model_name   >>>   {model_name}   ...   ')
    ## 读取 query 的 embedding 内容 
    with open(f'outslgb/encoded_questions_{model_name}_test.pkl', 'rb') as f:
        query_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)
    query_embs_concat.append(query_embs) 
    
    ## passage 的 embedding 内容 
    passages_embs_dir = {
        ## sentence model 
        'minilm': 'outslgb/encoded_passageJson_minilm.pkl',
        'minilmL12': 'outslgb/encoded_passageJson_minilmL12.pkl',
        ## 'paraphrase': 'outslgb/encoded_passageJson_paraphrase.pkl',
        ## model 
        'bge': 'outslgb/encoded_passageJson_bge.pkl',
        'gte': 'outslgb/encoded_passageJson_gte.pkl',
        'bgeM3': 'outslgb/encoded_passageJson_bgeM3.pkl',
    }[model_name]
    with open(passages_embs_dir, 'rb') as f:
        passages_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)
    passages_embs_concat.append(passages_embs) 

query_embs_concat = torch.hstack(query_embs_concat) 
passages_embs_concat = torch.hstack(passages_embs_concat) 

model_name   >>>   minilm   ...   
model_name   >>>   minilmL12   ...   
model_name   >>>   bge   ...   
model_name   >>>   gte   ...   
model_name   >>>   bgeM3   ...   


In [67]:
query_embs_concat.shape
passages_embs_concat.shape

torch.Size([3000, 3840])

torch.Size([466387, 3840])

In [68]:
## 计算得分 
model_name = 'concat'
for index in tqdm(unique_index, total=len(unique_index)): 
    retrival_pids = df_test.loc[df_test['index'] == index, 'id'].tolist() 
    scores = query_embs_concat[index] @ passages_embs_concat[retrival_pids].T
    df_test.loc[df_test['index'] == index, f'rank_score_{model_name}'] = scores.detach().cpu().numpy().astype(np.float32) 
    
df_test['rank_score_concat'] = df_test.groupby(by=['index'])['rank_score_concat'].transform(min_max_normalization)

  0%|          | 0/3000 [00:00<?, ?it/s]

In [69]:
### 0.1482
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_concat'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_concat.txt', sep='\t', index=False, header=None)

### 大模型 LLM 的计算得分 rank_score_llm_mistral

In [72]:
for model_name in ['Linq_mistral-NoClean', 'srf_mistral-NoClean', 
                   'Linq_mistral', 'srf_mistral', 
                   'Linq_mistral-WithAiresponse', 'srf_mistral-WithAiresponse', 
                   'Linq_mistral-WithKeywords', 'srf_mistral-WithKeywords', 
                   'Linq_mistral-WithKeywords-glm4', 'srf_mistral-WithKeywords-glm4', 
                  ]: 
    print(f'model_name   >>>   {model_name}   ...   ') 
    ## query-embs 
    with open(f'outslgb/encoded_question_{model_name}_test.pkl', 'rb') as f:
        query_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)
    
    ## passage-embs
    passage_dir = f'outslgb/encoded_passageJson_{model_name}_2048.pkl'
    if ('With' in passage_dir) or ('No' in passage_dir): 
        passage_dir = passage_dir.replace('-WithAiresponse', '').replace('-WithKeywords', '').replace('-NoClean', '').replace('-glm4', '')
    with open(passage_dir, 'rb') as f:
        passages_embs = torch.tensor(pickle.load(f), dtype=torch.float16, device=device)

    ## 计算得分 
    for index in tqdm(unique_index, total=len(unique_index)): 
        retrival_pids = df_test.loc[df_test['index'] == index, 'id'].tolist() 
        scores = query_embs[index] @ passages_embs[retrival_pids].T
        df_test.loc[df_test['index'] == index, f'rank_score_{model_name}'] = scores.detach().cpu().numpy().astype(np.float32)
    df_test[f'rank_score_{model_name}'] = df_test[f'rank_score_{model_name}'].astype(float) 

model_name   >>>   Linq_mistral-NoClean   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   srf_mistral-NoClean   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   Linq_mistral   ...   


...

  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   Linq_mistral-WithKeywords-glm4   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

model_name   >>>   srf_mistral-WithKeywords-glm4   ...   


  0%|          | 0/3000 [00:00<?, ?it/s]

In [73]:
### srf_mistral  0.1820
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_srf_mistral'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_srf_mistral.txt', sep='\t', index=False, header=None) 

In [74]:
### Linq_mistral 0.1824
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_Linq_mistral'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_Linq_mistral.txt', sep='\t', index=False, header=None) 

In [75]:
### srf_mistral-NoClean 
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_srf_mistral-NoClean'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_srf_mistral-NoClean.txt', sep='\t', index=False, header=None) 

### Linq_mistral-NoClean 
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_Linq_mistral-NoClean'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_Linq_mistral-NoClean.txt', sep='\t', index=False, header=None) 

In [76]:
### srf_mistral-WithAiresponse  0.1777
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_srf_mistral-WithAiresponse'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_srf_mistral-WithAiresponse.txt', sep='\t', index=False, header=None) 

### Linq_mistral-WithAiresponse  0.1771
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_Linq_mistral-WithAiresponse'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_Linq_mistral-WithAiresponse.txt', sep='\t', index=False, header=None) 

In [77]:
### srf_mistral-WithKeywords 0.1755
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_srf_mistral-WithKeywords'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_srf_mistral-WithKeywords.txt', sep='\t', index=False, header=None) 

### Linq_mistral-WithKeywords
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_Linq_mistral-WithKeywords'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_Linq_mistral-WithKeywords.txt', sep='\t', index=False, header=None) 

In [78]:
### srf_mistral-WithKeywords 0.1776
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_srf_mistral-WithKeywords-glm4'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_srf_mistral-WithKeywords-glm4.txt', sep='\t', index=False, header=None) 

### Linq_mistral-WithKeywords
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_Linq_mistral-WithKeywords-glm4'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20]))
test_candidates_list.to_csv('my_data_Linq_mistral-WithKeywords-glm4.txt', sep='\t', index=False, header=None) 

```python
'rank_score_srf_mistral'
'rank_score_Linq_mistral'

'rank_score_Linq_mistral-NoClean'
'rank_score_srf_mistral-NoClean', 

'rank_score_srf_mistral-WithAiresponse'
'rank_score_Linq_mistral-WithAiresponse'
'rank_score_srf_mistral-WithKeywords'
'rank_score_Linq_mistral-WithKeywords'
'rank_score_Linq_mistral-WithKeywords-glm4'
'rank_score_Linq_mistral-WithKeywords-glm4'
```

In [79]:
##  0.1829 (ori + ai_response) 
##  0.1834 (0.52*ori + 0.28*ai_response + 0.2*keywords) 
##  0.1838 (0.5*ori + 0.3*ai_response + 0.2*ai_response) 
##  0.1824 (0.5*ori + 0.3*ai_response + 0.2*WithKeywords) 
# ##  0.1839
df_test['rank_score_llm_merge'] = 0.5*(0.48*df_test['rank_score_srf_mistral'] + 0.52*df_test['rank_score_Linq_mistral']) + \
                0.5*(0.5*df_test['rank_score_srf_mistral-NoClean'] + 0.5*df_test['rank_score_Linq_mistral-NoClean']) + \
                0.5*(0.53*df_test['rank_score_srf_mistral-WithAiresponse'] + 0.47*df_test['rank_score_Linq_mistral-WithAiresponse']) + \
                0.5*(0.5*df_test['rank_score_srf_mistral-WithAiresponse'] + 0.5*df_test['rank_score_Linq_mistral-WithAiresponse']) 


# df_test['rank_score_llm_merge'] = 0.5*(0.48*df_test['rank_score_srf_mistral'] + 0.52*df_test['rank_score_Linq_mistral']) + \
#                 0.5*(0.5*df_test['rank_score_srf_mistral-WithAiresponse'] + 0.5*df_test['rank_score_Linq_mistral-WithAiresponse'])# + \
#                 # 0.2*(0.25*df_test['rank_score_srf_mistral-WithKeywords'] + 0.25*df_test['rank_score_Linq_mistral-WithKeywords'] + 0.28*df_test['rank_score_srf_mistral-WithKeywords-glm4'] + 0.28*df_test['rank_score_Linq_mistral-WithKeywords-glm4'])

# ## 
# df_test['rank_score_llm_merge'] = 0.3*(0.48*df_test['rank_score_srf_mistral'] + 0.52*df_test['rank_score_Linq_mistral']) + \
#                 0.3*(0.5*df_test['rank_score_srf_mistral-NoClean'] + 0.5*df_test['rank_score_Linq_mistral-NoClean']) + \
#                 0.2*(0.5*df_test['rank_score_srf_mistral-WithAiresponse'] + 0.5*df_test['rank_score_Linq_mistral-WithAiresponse'])


In [80]:
### merged 
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_llm_merge'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list)
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20])) 
test_candidates_list.to_csv('my_data_llm_merge-noclean-airesponse.txt', sep='\t', index=False, header=None) 

## 融合分数 
```
得分 = 0.18006 : 0.18*concat + 0.4*srf-mistral + 0.42*linq-mistral
```

In [81]:
# 直接使用 numpy 进行向量化计算
weights = np.array([0.1, 0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]) 
np.sum(weights)
q2p_scores_cols = [# 'rank_score_mean',            ### 0.14840
                   # 'rank_score_weights',         ### 0.14827
                    'rank_score_concat',          ### 0.14862
                    'rank_score_srf_mistral',     ### 0.18041
                    'rank_score_Linq_mistral',    ### 0.18217
                    # 'rank_score_llm_merge', 
                    'rank_score_srf_mistral-WithAiresponse',
                    'rank_score_Linq_mistral-WithAiresponse',
                    'rank_score_srf_mistral-WithKeywords',
                    'rank_score_Linq_mistral-WithKeywords',
                    'rank_score_srf_mistral-WithKeywords-glm4',
                    'rank_score_Linq_mistral-WithKeywords-glm4'
                  ] 
# 确保数据类型正确，避免计算时出现类型错误
df_test[q2p_scores_cols] = df_test[q2p_scores_cols].astype(float) 
# 计算加权得分
df_test['rank_score_overall'] = np.sum(df_test[q2p_scores_cols].values * weights, axis=1) / np.sum(weights) 

1.0

In [82]:
### llm-overall 
df_test['rank_score_overall'] = df_test['rank_score_overall'].astype(float) 
test_candidates_list = df_test.sort_values(by=['index', 'rank_score_overall'], 
                                           ascending=[True, False]).groupby(by=['index'], as_index=True)['pids'].apply(list) 
test_candidates_list = test_candidates_list.apply(lambda x: ','.join(x[:20])) 
test_candidates_list.to_csv('my_data_overall.txt', sep='\t', index=False, header=None) 

In [83]:
1557550080 ## qwen1.5-14b-gptq
9399951360 ## glm4-9b

1557550080

9399951360

In [85]:
print('Finish !!!')

Finish !!!
